# Pygame Notes!!! 
(see https://inventwithpython.com/pygame/chapter2.html for a background check) 

In [1]:
import pygame, sys
from pygame.locals import *

pygame 2.0.1 (SDL 2.0.14, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


Very quick here is how you will generally want to animate in pygame. You want to draw a screen and then change that screen every frame to create the illusion of movement. To do this every frame you will want to fill in the background to erase your previous drawing, (using SURFACE.fill() => more on that later) and then draw your stuff. You can draw primitive types which will be shown below or use the blit method to draw other surfaces onto your game surface. blit(img, center) takes in the img which can be an image file or a surface and then center which can be a tuple of 2 integers, the x and y coordinates of the center, or it can be a Rect object, in which case Rect.center will be used. Also coordinate systems work by placing (0,0) at the top left of the screen and then increasing as you go down or right

### Dealing with input

There are two main ways to deal with processing input, Event Handling, and State Checking. We will go through each of these one at a time, starting off with Event Handling. With Event Handling pygame creates an list of what has happened and allows you to process this, usually once per frame. The list of events is known as the Event Queue and also keeps the order of events. Each individual action is stored as a Event object. To access all the Events in the Event Queue you need to use the pygame.event.get() function which returns a list of all Events in the Event Queue and then removes those Events from the Event Queue. Each event has a specific id named in an attribute called type. After checking the type you can then access some more attributes based on the type. We will now list a some types and attributes associated with them:

1. MOUSEMOTION
    - pos => returns a tuple with the x and y positions of the mouse
    - rel => returns a tuple with the difference of the x and y coordinates since the last call
    - buttons => returns a tuple of three values representing the the state of each mouse button during this move
2. MOUSEBUTTONUP
    - pos => the same
    - button => number (0 or 1) representing whether the button is pressed or released  
3. MOUSEBUTTONDOWN
    - pos => the same
    - button => same
4. KEYDOWN
    - key => returns a pygame.K_ object which describes what key is pressed
    - mod => returns a number (0 or 1) representing whether modifier keys are not pressed or pressed
    - unicode => returns a unicode string representing the key press. Will return empty string if key is not something that be expressed in unicode. Example of use is to translate words (unicode deals with modifers like the shift key automatically)
5. KEYUP 
    - key => same
    - mod => same

In [3]:
pygame.init()
pygame.display.set_caption("Dealing with input by Handing Events")
gameDisplay = pygame.display.set_mode((400,600))

run = True
while run:  
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
            
        elif event.type == KEYDOWN: #Try typing capital letters and see what happens below!
            print(event.unicode)
            
            if event.key == pygame.K_SPACE:
                run = False
        
        elif event.type == MOUSEMOTION:
            mousecoor = event.pos 
            gameDisplay.fill((0,0,0))
            pygame.draw.circle(gameDisplay, (255,0,255), mousecoor, 20) #You will learn this later, for now notice the mousecoor
            
    pygame.display.update()
pygame.quit()

Now we will look at state checking. With state checking all you do is call a function to see the state of an input device. This is usually the easiest way to find out what is going on since you can immediately know what the user is doing to a certain key.
Each input device has an object that provides several methods to check the current state. Here are some:

1. pygame.mouse => A module that contains functions for accessing the mouse. The module can also change the location of the mouse
    - get_pos() => returns a tuple with the x,y coordinates of the mouse position
    - get_rel() => returns the difference in x,y since the last time this function was called
    - get_pressed() => returns a tuple with three values, 0 or 1, not pressed or pressed, for each of the three mouse buttons.
    
2. pygame.key => A module that contains functions for accessing the keyboard. It can control repeat rates and translate key ids into English values
    - get_pressed() => returns a tuple containing the pressed state for every key on the key board. To index this you can use they id values
    - get_mods() => returns an integer representing whether a modifer key is pressed or not. The integer itself is a bitwise array of ecah key, using their id values. To test if a combination of keys is pressed use the python bitwise operators with the modifier keys names
    
State Checking however does have a couple of flaws. The first being that you have no idea what order of action ensued because when calling the function three buttons could be pressed at once with you having no idea which is for which. Also, you can just miss button presses. If the user clicks very fast, between the calls to check the button state, then you will never know. This is why Event Handling exists, to solve these issues.

Something to keep in mind is that the values of these function can switch while you are computing a frame, so rather than computing it multiple times, store the function output in a variable and check that variable.

In [21]:
# Don't worry about the references to drawing shapes. You will learn that later. For now just watch

pygame.init()
pygame.display.set_caption("Dealing with inputs by checking states")
gameDisplay = pygame.display.set_mode((400,600))
pygame.time.delay(2000)

run = True
while run:
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
            
    pmods = pygame.key.get_mods()
    
    if pygame.key.get_pressed()[pygame.K_SPACE]:
        gameDisplay.fill((0,0,0))
        pygame.draw.circle(gameDisplay, (255,255,255), (100,100), 20)
        pygame.time.delay(500)
        run = False
    
    elif pygame.mouse.get_rel() != (0,0):
        gameDisplay.fill((0,0,0))
        pygame.draw.circle(gameDisplay, (0,0,255), pygame.mouse.get_pos(), 20)
        
    elif pmods & pygame.KMOD_ALT:
        gameDisplay.fill((0,0,0))
        pygame.draw.rect(gameDisplay, (0,255,0), (190,290,20,20))
        
    elif pmods == 0:
        pygame.draw.rect(gameDisplay, (255,0,0), (250,0,20,20))
    
    pygame.display.update()
    
pygame.quit()
    

### Dealing with showing text:

Pygame works by drawing text onto a new surface to create an image of the text. Then it blits this onto the main surface. 

There are 7 basic steps:
1. Create a DisplaySurface Object for the screen => use display.set_mode(X,Y) this will create screen with dimensions X,Y
2. Create a Font object => use font.Font('Font file name', X) this will create font object with font 'Font name' and size X or use font.SysFont('String name of font', X) which will create a font from your system with size X
3. Create a TextSurface Object for the text to go on => use the render() method of the font object. render takes in four parameters. render('Text', antialias, color, background = none), 'Text' is what you want to display, antialias is a boolean that when set to True blurs pixels at the edges to create smoother lines, color is the color of the text, and background is the highlight color defaulted to nothing
4. Create a rectangular object for the text surface => use get_rect() of TextSurface Object
5. Set the position of the rectangular object => set attribute center to a tuple of two values, xposition and yposition
6. Copy TextSurface object onto DisplaySurface Object => use blit(text, textRect) where text is the TextSurface object and textRect is the rectangular object
7. Show display surface object on screen => display.update() or display.flip()

Example shown below:

*Side note: All fonts your system has can be found using pygame.font.get_fonts()

In [1]:
pygame.init()
pygame.display.set_caption("Displaying Text")

#Step 1
gameDisplay = pygame.display.set_mode((400,400))
#Step 2
text_font = pygame.font.Font("freesansbold.ttf", 40)
#Step 3
textDisplaySurface = text_font.render("Example Here!", True, (255,0,0))
#Step 4
text_rect = textDisplaySurface.get_rect()
#Step 5
text_rect.center = (200,200)
#Step 6
gameDisplay.blit(textDisplaySurface, text_rect)
#Step 7
pygame.display.flip()

pygame.time.wait(5000)
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Although usually you want to create some function that will do this for you:
def display_words(gameDisplay, font, text_size, text, antialias, color, center_xcoor, center_ycoor):
    #Steps 2-6
    text_font = pygame.font.Font(font, text_size)
    textDisplaySurface = text_font.render(text, antialias, color)
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (center_xcoor, center_ycoor)
    gameDisplay.blit(textDisplaySurface, text_rect)

    
pygame.init()
pygame.display.set_caption("Displaying Text 2")
gameDisplay = pygame.display.set_mode((400,400))
display_words(gameDisplay, "freesansbold.ttf", 40, "Example Here!", True, (255,0,0), 200,200)
pygame.display.update()
pygame.time.wait(4000)
#Have to do way less steps!
display_words(gameDisplay, "freesansbold.ttf", 20, "Example Here Too!", True, (0,255,0), 200,325)
pygame.display.update()
pygame.time.wait(4000)
pygame.quit()


### Dealing with drawing shapes

Here we will talk about how to get your shapes on screen. As a general note, pygame draws shapes directly onto the surface object.

First and foremost we will talk about rectangles. To represent a rectangle in Pygame you can just have a tuple of four integers with the values you will see at the bottom but you probably want to create a pygame.Rect object. To do this you use .Rect(x, y, w, h) where x,y is the x-coordinate and y-coordinate of the TOP LEFT CORNER, and w,h is the width and height in pixels respectively. The good thing is that you can calculate other points instantly and reassign them as you want by using the Rect object's attributes. Here they are listed out: (left, right, top, bottom, centerx, centery, width, height, size, topleft, topright, bottomright, bottomleft, midleft, midright, midtop, midbottom) To find out more info just type the attribute and then press shift+tab. The Rect object will come in much more handy soon when dealing with collsion detection.

Example shown here:

In [10]:
pygame.init()
pygame.display.set_caption("Rectangle attributes")
gameDisplay = pygame.display.set_mode((400,400))
myRect = pygame.Rect(200,200,20,20)
print(myRect.topleft)
myRect.centerx = 200
myRect.centery = 200
print(myRect.topleft)
pygame.quit()

(200, 200)
(190, 190)


Now we get to drawing shapes. When drawing shapes to the screen it is known as drawing primitives. These shapes include. rectangles, circles, ellipses, lines, or individual pixels. I will now list out a bunch of functions and their purposes:

1. fill(c) is a method of you surface object. It will fill the entire screen with whatever color you choose to put for c.
2. pygame.draw.polygon(surface, color, pointlist, width) will draw a polygon. You first pass in your sruface object. Then the color. Pointlist is a tuple or list of all your points. The method will connect the a point with the next point in the list until it reaches the last one, to which it connects the first one. (So order does matter!). Finally, width is optional. If you pass in nothing it will fill the polygon, if you pass in an number it will draw the edges with that thickness
3. pygame.draw.line(surface, color, start_point, end_point, width) will draw a line between your start_point and end_point coordinates. The rest is similar or self explanatory.
4. pygame.draw.lines(surface, color, closed, pointlist, width) is basically like #2 except if you pass True for closed it will connect the first and last points in pointlist while if you pass False it won't. 
5. pygame.draw.circle(surface, color, center_point, radius, width) will draw a circle. Everything else is self-explanatory yet again.
6. pygame.draw.ellipse(surface, color, bounding_rectangle, width) will draw an ellipse within the bounding_rectangle which can be a rectangle object or just a tuple with the four necessary values. 
7. pygame.draw.rect(surface, color, rectangle_tuple, width) will draw a rectangle. The rectangle_tuple can be a tuple or a Rect object

You must also understand PixelArray Objects. They are the best way to draw certain colors to individual pixels. (The other way is misusing a method above. See if you can figure it out!) To start you must create a pygame.PixelArray object by using pygame.PixelArray(surface). By doing this you will also lock the surface (you can check whether a surface is locked or not using the get_locked() method) which means you cannot use the blit() method to draw PNG or JPG images on the surface. After that you simply use pixObj[x][y] = color and that will set the pixel at (x,y) to your assigned color. When you are done just delete the pixObj object to unlock the surface object. 

Below is an example with everything shown:

In [4]:
import pygame, sys
from pygame.locals import *

pygame.init()
DISPLAYSURF = pygame.display.set_mode((500, 400), 0, 32)
pygame.display.set_caption('Drawing')

BLACK = (  0,   0,   0)
WHITE = (255, 255, 255)
RED = (255,   0,   0)
GREEN = (  0, 255,   0)
BLUE = (  0,   0, 255)

DISPLAYSURF.fill(WHITE)

pygame.draw.polygon(DISPLAYSURF, GREEN, ((146, 0), (291, 106), (236, 277), (56, 277), (0, 106)))

pygame.draw.line(DISPLAYSURF, BLUE, (60, 60), (120, 60), 4)
pygame.draw.line(DISPLAYSURF, BLUE, (120, 60), (60, 120))
pygame.draw.line(DISPLAYSURF, BLUE, (60, 120), (120, 120), 4)

pygame.draw.circle(DISPLAYSURF, BLUE, (300, 50), 20, 0)
pygame.draw.ellipse(DISPLAYSURF, RED, (300, 250, 40, 80), 1)

pygame.draw.rect(DISPLAYSURF, RED, (200, 150, 100, 50))

pixObj = pygame.PixelArray(DISPLAYSURF)
pixObj[480][380] = BLACK
pixObj[482][382] = BLACK
pixObj[484][384] = BLACK
pixObj[486][386] = BLACK
pixObj[488][388] = BLACK
del pixObj

pygame.display.update()
pygame.time.wait(10000)
pygame.quit()

### Dealing with Time 

To keep track a good track of time you can use the pygame.time module. Pygame keeps track of time in milliseconds. Know this. 

Here will be a list of things to know, visit this website for more https://www.pygame.org/docs/ref/time.html. The examples are below

1. pygame.time.get_ticks() => will return the amount of time since pygame.init() was called, you can use it to keep track of things like the amount of seconds that have passed
2. pygame.time.wait(milliseconds) => will sleep the program for the number of milliseconds that you put inside. A program that waits for even a small number of milliseconds will help the process greatly. This however is slightly less accurate than the next function
3. pygame.time.delay(milliseconds) => will pause the program for the number of milliseconds that you put inside. This method will use the processor in order to make the delay more accurate.
4. pygame.time.set_timer(eventid, milliseconds, once = False) will call the eventid every time the number of milliseconds you specify happens. The first event will begin after that number of milliseconds has passed. eventid is actually a number but it should be between the pygame values pygame.USEREVENT and pygame.NUMEVENTS. You can check for this event from the event queue
5. pygame.time.Clock => module to help keep track of time. Can also keep control of framerate. Methods will be now be listed. 
    - pygame.time.Clock.tick(fr = 0) => should be called once per frame. It will return how many milliseconds have passed since prevous call. The more important function is that if you pass in a framerate then it will delay the program to make sure that your program will never run more than fr frames per second. This function is not as accurate as possible since it is CPU aware. To find a CPU expensive more accurate function visit the docs. 
    - pygame.time.Clock.get_time() => will return the amount of time in milliseconds that has passed between the last two calls to tick()
    - pygame.time.Clock.get_rawtime() will do the same thing as get_time() but subtract the time tick() was delaying due to the framerate
    - pygame.time.Clock.get_fps() will return a float representing the framerate that will be found by averaging the last ten calls to tick()


In [4]:
pygame.init()
pygame.display.set_caption("Dealing with time")
gameDisplay = pygame.display.set_mode((400,600))

def display_time(gameDisplay, time):
    text_font = pygame.font.SysFont("arial", 40)
    textDisplaySurface = text_font.render(time, True, (0,255,255))
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (200, 300)
    gameDisplay.blit(textDisplaySurface, text_rect)

def my_event(gameDisplay):
    text_font = pygame.font.Font("freesansbold.ttf", 20)
    textDisplaySurface = text_font.render("My Event is happening right now!", False, (0,255,255))
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (200, 500)
    gameDisplay.blit(textDisplaySurface, text_rect)
    pygame.display.update()
    pygame.time.wait(1000)

run = True

start_ticks = pygame.time.get_ticks()

clock = pygame.time.Clock()

myEvent = pygame.USEREVENT+1 
pygame.time.set_timer(myEvent, 2000)

while run:
    
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
        
        elif event.type == myEvent:
            my_event(gameDisplay)
        
        elif event.type == KEYDOWN:
            if event.key == pygame.K_SPACE:
                run = False
            elif event.key == pygame.K_r:
                start_ticks = pygame.time.get_ticks()
            elif event.key == pygame.K_s:
                pygame.time.set_timer(myEvent, 0)
                
    seconds_passed = round((pygame.time.get_ticks()-start_ticks)/1000, 2)
    
    gameDisplay.fill((0,0,0))
    display_time(gameDisplay, str(seconds_passed))
    pygame.display.update()
    clock.tick(60)
            
        
pygame.quit()
            


### Dealing with collision with primitives

Small Sidenote, if you are creating a game with player classes then you will want to have all the player classes override a pygame class named pygame.sprite.Sprite. You may also want to create a rect object hitbox associated with it. There are lots of other complicated things you could do however we are just dealing with primitives so we will let that go for now. Otherwise there is a lot more to this.

When checking for collision between primitives you could do this by creating your own checking function or using pygame's rectangle collision functionality. With shapes like circles or other things you can create a rectangle hitbox associated with it. With pygame you will use RECTOBJECT.colliderect(OTHERRECTOBJECT). When creating you own method you can get creative and check something like: are all four corners of the rectangle outside the other rectangle?

Here is an example using colliderect.

In [3]:
pygame.init()

rect1 = pygame.Rect(100,100,20,20)
rect2 = pygame.Rect(100,90,20,20)
rect3 = pygame.Rect(300,300,20,20)

if rect1.colliderect(rect2):
    print("collision between rect 1 and 2")
if rect1.colliderect(rect3):
    print("collision between rect 1 and 3")
if rect2.colliderect(rect3):
    print("collision between rect 2 and 3")
    
pygame.quit()

collision between rect 1 and 2


Now a small game using what we have just learned

In [3]:
pygame.init()
pygame.display.set_caption("Small game")
gameDisplay = pygame.display.set_mode((500,500))

def display_message(gameDisplay, text):
    text_font = pygame.font.Font("freesansbold.ttf", 30)
    textDisplaySurface = text_font.render(text, False, (0,255,255))
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (250, 250)
    gameDisplay.blit(textDisplaySurface, text_rect)
    pygame.display.update()
    
def boundry_collision(vector):
    if int(vector[0]) < 10 or int(vector[0]) > 490 or int(vector[1]) < 10 or int(vector[1]) > 490:
        return True
    return False

def square_collision(botvector, playervector):
    bot_hitbox = pygame.Rect(int(botvector[0]), int(botvector[1]), 20, 20)
    player_hitbox = pygame.Rect(int(playervector[0]), int(playervector[1]), 20, 20)
    
    if bot_hitbox.colliderect(player_hitbox):
        return True
    return False

def get_movement_vector(botvector, playervector):
    difference = playervector - botvector
    bot_movement_vector = pygame.math.Vector2(0,0)
    
    if difference[0] < 0:
        bot_movement_vector += pygame.math.Vector2(-1,0)
    elif difference[0] == 0:
        bot_movement_vector += pygame.math.Vector2(0,0)
    elif difference[0] > 0:
        bot_movement_vector += pygame.math.Vector2(1,0)
        
    if difference[1] < 0:
        bot_movement_vector += pygame.math.Vector2(0,-1)
    elif difference[1] == 0:
        bot_movement_vector += pygame.math.Vector2(0,0)
    elif difference[1] > 0:
        bot_movement_vector += pygame.math.Vector2(0,1)
        
    bot_movement_vector.normalize_ip()
    return bot_movement_vector

def display_strats(strat, center_x, center_y):
    textfont = pygame.font.Font('freesansbold.ttf', 12)
    textSurface = textfont.render(strat, True, (0,255,0))
    textRect = textSurface.get_rect()
    textRect.center = (center_x, center_y)
    gameDisplay.blit(textSurface, textRect)

dif_type_speed = True

if dif_type_speed:
    init_bot_buffer_time = 1000
    min_bot_buffer_time = 500
    init_bot_speed = 2.5
else:
    init_bot_buffer_time = 500
    min_bot_buffer_time = 300
    init_bot_speed = 2


botCalc = pygame.USEREVENT+1 
difIncrease = pygame.USEREVENT+2
bot_buffer_time = init_bot_buffer_time

pygame.time.set_timer(botCalc, bot_buffer_time)
pygame.time.set_timer(difIncrease, 5000)

clock = pygame.time.Clock()
player_pos = pygame.math.Vector2(250,400)
bot_pos = pygame.math.Vector2(250,10)
player_speed = 3
bot_speed = init_bot_speed
bot_movement_vector = pygame.math.Vector2(0,0)
start_ticks = pygame.time.get_ticks()
run = True
while run:
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
        if event.type == KEYDOWN:
            if event.key == pygame.K_SPACE:
                run = False
        if event.type == botCalc:
            #Move bot through a simple algorithm to get as close as possible
            bot_movement_vector = get_movement_vector(bot_pos, player_pos) 
        if event.type == difIncrease:
            display_message(gameDisplay, "Difficulty Increase!")
            pygame.time.wait(500)
            if bot_buffer_time >= min_bot_buffer_time:
                bot_buffer_time -= 50    
            pygame.time.set_timer(botCalc, bot_buffer_time)
            bot_speed += 0.1
                
    
    #Move player through key presses
    pkeys = pygame.key.get_pressed()

    player_movement_vector = pygame.math.Vector2(0,0)
    if pkeys[pygame.K_UP]:
        player_movement_vector += pygame.math.Vector2(0,-1)
    if pkeys[pygame.K_LEFT]:
        player_movement_vector += pygame.math.Vector2(-1,0)
    if pkeys[pygame.K_DOWN]:
        player_movement_vector += pygame.math.Vector2(0,1)
    if pkeys[pygame.K_RIGHT]:
        player_movement_vector += pygame.math.Vector2(1,0)
    
    if player_movement_vector != pygame.math.Vector2(0,0):
        player_movement_vector.normalize_ip()
        
    #Update positions
    player_pos += player_movement_vector*player_speed
    bot_pos += bot_movement_vector*bot_speed
        
    #Check if collision with boundry exists. If it does, perform cleanup
    if boundry_collision(player_pos):
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "You have crashed into the wall!")
        pygame.time.wait(2000)
        player_pos = pygame.math.Vector2(250,250)
        bot_pos = pygame.math.Vector2(250,10)
        bot_buffer_time = init_bot_buffer_time
        bot_speed = init_bot_speed
        pygame.time.set_timer(botCalc, bot_buffer_time)
        pygame.time.set_timer(difIncrease, 5000)
        start_ticks = pygame.time.get_ticks()
        
    #Make sure bot is at the right place. 
    if boundry_collision(bot_pos):
        bot_movement_vector = get_movement_vector(bot_pos, player_pos)
    
    #Check if collision between squares exists. If it does, perform cleanup
    if square_collision(bot_pos, player_pos):
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "You have been caught by the bot!")
        pygame.time.wait(2000)
        player_pos = pygame.math.Vector2(250,250)  
        bot_pos = pygame.math.Vector2(250,10)
        bot_buffer_time = init_bot_buffer_time
        bot_speed = init_bot_speed
        pygame.time.set_timer(botCalc, bot_buffer_time)
        pygame.time.set_timer(difIncrease, 5000)
        start_ticks = pygame.time.get_ticks()
        
    gameDisplay.fill((0,0,0))
    #The reason for the -10 is that the rectangles are drawn from the top left corner but the vectors are the center coordinates
    pygame.draw.rect(gameDisplay, (0,255,255), (int(player_pos[0])-10, int(player_pos[1])-10, 20,20))
    pygame.draw.rect(gameDisplay, (255,0,0), (int(bot_pos[0])-10, int(bot_pos[1])-10, 20,20))
    display_strats("The time so far is: " + str(round((pygame.time.get_ticks()-start_ticks)/1000, 2)), 400, 25)
    display_strats("The bot buffer time is: " + str(bot_buffer_time), 400, 50)
    display_strats("The bot movement speed is: " + str(round(bot_speed, 1)), 400, 75)
    pygame.display.update()
    clock.tick(60)

pygame.quit()

Here is the game but improved (using player classes and so on). This is the version of the game that is used to help create the NN platform.

In [8]:
import pygame
from pygame.locals import *

class wall:
    
    def __init__(self, center_xcoor, center_ycoor, width, height):
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(center_xcoor-width/2,center_ycoor-height/2,width,height)
        self.pos = pygame.math.Vector2(center_xcoor, center_ycoor)
        
    
    def show(self, gameDisplay):
        pygame.draw.rect(gameDisplay, (255,255,255), self.hitbox)

class bot:
    
    def __init__(self, buffer_time =500, min_buffer_time =300, speed=2, pos =pygame.math.Vector2(400,10), width =20, height =20):
        
        self.buffer_time = buffer_time
        self.min_buffer_time = min_buffer_time
        self.speed = speed
        self.pos = pos
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(pos[0]-width/2, pos[1]-height/2, width, height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
    def show(self, gameDisplay):
        rect = (int(self.hitbox.topleft[0]), int(self.hitbox.topleft[1]), self.width, self.height)
        pygame.draw.rect(gameDisplay, (255,0,0), rect)
        
    def get_movement(self, player):
        
        difference = player.pos - self.pos

        if difference[0] < 0:
            self.movement_vector += pygame.math.Vector2(-1,0)
        elif difference[0] == 0:
            self.movement_vector += pygame.math.Vector2(0,0)
        elif difference[0] > 0:
            self.movement_vector += pygame.math.Vector2(1,0)
        
        if difference[1] < 0:
            self.movement_vector += pygame.math.Vector2(0,-1)
        elif difference[1] == 0:
            self.movement_vector += pygame.math.Vector2(0,0)
        elif difference[1] > 0:
            self.movement_vector += pygame.math.Vector2(0,1)
        
        if self.movement_vector != pygame.math.Vector2(0,0):
            self.movement_vector.normalize_ip()
    
    def update(self):
        self.pos += self.movement_vector*self.speed
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
    
    def reset(self):
        self.buffer_time = 500
        self.min_buffer_time = 300
        self.speed = speed = 2
        self.pos = pygame.math.Vector2(400,10)
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
class player:
    
    def __init__(self, speed =3, pos =pygame.math.Vector2(400,500), width =20, height =20):
        self.speed = speed
        self.pos = pos
        self.width = width
        self.height = height
        self.hitbox = pygame.Rect(pos[0]-width/2, pos[1]-height/2, width, height)
        self.movement_vector = pygame.math.Vector2(0,0)
        
    def show(self, gameDisplay):
        rect = (int(self.hitbox.topleft[0]), int(self.hitbox.topleft[1]), self.width, self.height)
        pygame.draw.rect(gameDisplay, (0,0,255), rect)
        
    def get_movement(self, pkeys):
        
        if pkeys[pygame.K_UP]:
            self.movement_vector += pygame.math.Vector2(0,-1)
        if pkeys[pygame.K_LEFT]:
            self.movement_vector += pygame.math.Vector2(-1,0)
        if pkeys[pygame.K_DOWN]:
            self.movement_vector += pygame.math.Vector2(0,1)
        if pkeys[pygame.K_RIGHT]:
            self.movement_vector += pygame.math.Vector2(1,0)
        if self.movement_vector != pygame.math.Vector2(0,0):
            self.movement_vector.normalize_ip()
            
    def update(self):
        self.pos += self.movement_vector*self.speed
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)
    
    def reset(self):
        self.speed = 3
        self.pos = pygame.math.Vector2(400,500)
        self.hitbox = pygame.Rect(self.pos[0]-self.width/2, self.pos[1]-self.height/2, self.width, self.height)
        self.movement_vector = pygame.math.Vector2(0,0)

def collision(square1, square2):
    return square1.hitbox.colliderect(square2.hitbox)  

def display_message(gameDisplay, text):
    text_font = pygame.font.Font("freesansbold.ttf", 30)
    textDisplaySurface = text_font.render(text, False, (0,255,255))
    text_rect = textDisplaySurface.get_rect()
    text_rect.center = (400, 300)
    gameDisplay.blit(textDisplaySurface, text_rect)
    pygame.display.update()
    
def boundry_collision(square):
    return (int(square.pos[0]) < 0+square.width/2 or int(square.pos[0]) > 800-square.width/2 or int(square.pos[1]) < 0+square.height/2 or int(square.pos[1]) > 600-square.height/2)
    
def display_strats(strat, center_x, center_y):
    textfont = pygame.font.Font('freesansbold.ttf', 12)
    textSurface = textfont.render(strat, True, (0,255,0))
    textRect = textSurface.get_rect()
    textRect.center = (center_x, center_y)
    gameDisplay.blit(textSurface, textRect)

def reset(bot, player):
    global start_ticks
    start_ticks = pygame.time.get_ticks()
    bot.reset()
    pygame.time.set_timer(botCalc, bot.buffer_time)
    pygame.time.set_timer(difIncrease, 5000)
    player.reset()
    

#Pygame Variables
pygame.init()
pygame.display.set_caption("NN ATTEMPT BOIS")
gameDisplay = pygame.display.set_mode((800,600))
clock = pygame.time.Clock()

#Level Variables
player = player()
bot = bot()
wall = wall(400,300,50,50)

botCalc = pygame.USEREVENT+1 
pygame.time.set_timer(botCalc, bot.buffer_time)

difIncrease = pygame.USEREVENT+2
pygame.time.set_timer(difIncrease, 5000)

#Starting Variables
start_ticks = pygame.time.get_ticks()
run = True
while run:
    for event in pygame.event.get():
        if event.type == QUIT:
            run = False
        elif event.type == KEYDOWN:
            if event.key == pygame.K_SPACE:
                run = False
        elif event.type == botCalc:
            bot.get_movement(player)
        elif event.type == difIncrease:
            display_message(gameDisplay, "Difficulty Increase!")
            pygame.time.wait(500)
            if bot.buffer_time >= bot.min_buffer_time:
                bot.buffer_time -= 50    
            pygame.time.set_timer(botCalc, bot.buffer_time)
            bot.speed += 0.1
    
    #Get the movements from all and update positions for all
    player.get_movement(pygame.key.get_pressed())
    player.update()
    bot.update()  
    
    #Boundry logic
    #First check collision with boundries
    if boundry_collision(bot):
        bot.get_movement(player)
    if boundry_collision(player):
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "You have crashed into the wall!")
        pygame.time.wait(2000)
        reset(bot, player)
    #Then check collision between all three objects
    if collision(bot, wall):
        difference = bot.pos - wall.pos
        total_width = bot.width + wall.width 
        total_height = bot.height + wall.height
        if difference[0] < 0:
            movex = -abs(difference[0] - total_width)
        else:
            movex = abs(difference[0] - total_width)
        if difference[1] < 0:
            movey = -abs(difference[1] - total_height)
        else:
            movey = abs(difference[1] - total_height)
        
        bot.movement_vector = pygame.math.Vector2(movex, movey)
        bot.movement_vector.normalize_ip()
        bot.movement_vector *= 2
        
    if collision(bot, player):
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "You have been caught by the bot!")
        pygame.time.wait(2000)
        reset(bot, player)
        
    if collision(player, wall):
        gameDisplay.fill((0,0,0))
        display_message(gameDisplay, "Don't crash into a wall!")
        pygame.time.wait(1500)
        reset(bot, player)        

    #Visuals
    gameDisplay.fill((0,0,0))
    bot.show(gameDisplay)
    player.show(gameDisplay)
    wall.show(gameDisplay)
    display_strats("The time so far is: " + str(round((pygame.time.get_ticks()-start_ticks)/1000, 2)), 700, 25)
    display_strats("The bot buffer time is: " + str(bot.buffer_time), 700, 50)
    display_strats("The bot movement speed is: " + str(round(bot.speed, 1)), 700, 75)
    pygame.display.update()
    clock.tick(60)
            
pygame.quit()

KeyboardInterrupt: 

In [9]:
pygame.quit()